Used tutorial https://github.com/maobedkova/TopicModelling_PySpark_SparkNLP/blob/master/Topic_Modelling_with_PySpark_and_Spark_NLP.ipynb for reference

In [1]:
import sparknlp
from sparknlp.annotator import Tokenizer, PerceptronModel
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA
from pyspark.sql import types as T

spark = sparknlp.start()

24/11/10 16:01:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
og_data = spark.read.csv("../data_topicmodel.csv", header= True).select(["id", "cleaned_text"])

In [3]:
#Remove sample
og_data.show(5)
sample_data = og_data.sample(0.001)

+-----+--------------------+
|   id|        cleaned_text|
+-----+--------------------+
|hk5r2|i had an appointm...|
|iqimz|i created this si...|
|pfzt5|hello everyone  i...|
|pk714|i grew up with bo...|
|q0q8x|i have to ask whe...|
+-----+--------------------+
only showing top 5 rows



In [4]:
documentAssembler = DocumentAssembler()\
     .setInputCol("cleaned_text")\
     .setOutputCol('document')

In [5]:
tokenizer = Tokenizer() \
            .setInputCols(['document'])\
            .setOutputCol('tokenized')

In [6]:
normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') 

In [7]:
lemmatizer = LemmatizerModel.load("../models/lemma_ewt_en_3.4.3_3.0_1651416655397/")\
      .setInputCols("normalized")\
      .setOutputCol("lemmatized")

In [8]:
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "rselfimprovement"]

topic_specific = ["self", "improvement", "change", "action",
    'change', 'start', 'goal', 'habit', 'new', 'old', 
    'care', 'world', 'everyone', 'love', 'u', 'right', 'mean', 'matter',
    'best', 'step', 'focus', 'hard', 'small',
    'bad', 'help', 'time', 'problem', 'issue', 'advice',
    'bit', 'experience', 'different',
    'point', 'situation', 'negative', 'control', 'positive',
    'use', 'question', 'idea', 'amp', 'medium', 'hour', 'day', 'minute',
    'aaaaloot']

stopwords = english + time + reddit + topic_specific

from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['normalized']) \
     .setOutputCol('unigrams') \
     .setStopWords(stopwords)

In [9]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['normalized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [10]:
pos = PerceptronModel.load("/project/macs40123/spark-jars/pos_anc_en_3.0.0_3.0_1614962126490/")\
      .setInputCols("document", "normalized")\
      .setOutputCol("pos")

In [11]:
from sparknlp.base import Finisher

finisher = Finisher().setInputCols(['unigrams', 'ngrams', 'pos'])

In [12]:
my_pipeline = Pipeline(
      stages = [
          documentAssembler,
          tokenizer,
          normalizer,
          stopwords_cleaner,
          ngrammer,
          pos,
          finisher
      ])

In [13]:
pipelineModel = my_pipeline.fit(sample_data)
processed_data = pipelineModel.transform(sample_data)
processed_data.show(1)

+------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos|
+------+--------------------+--------------------+--------------------+--------------------+
|1to3a6|can people sugges...|[suggest, courses...|[can, people, sug...|[MD, NNS, VB, NNS...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [14]:
processed_data.columns

['id', 'cleaned_text', 'finished_unigrams', 'finished_ngrams', 'finished_pos']

In [15]:
#data.select('document').show(5, truncate = 100)

In [16]:
#data.select('tokenized').show(1, truncate = 100)

In [17]:
#data.select('normalized').show(1, truncate = 100)

In [18]:
#data.select('lemmatized').show(1, truncate = 1000)

In [19]:
processed_data.select('finished_unigrams').show(10, truncate = 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                   finished_unigrams|
+----------------------------------------------------------------------------------------------------+
|[suggest, courses, one, quick, courses, meaning, hours, improve, ones, worth, heres, couple, firs...|
|[already, come, long, super, shy, borderline, shutin, guy, highschool, obvious, boring, actually,...|
|[accomplished, hours, worth, coursework, first, awake, apart, biology, lab, lecture, done, else, ...|
|[guys, seems, flake, urban, dictionary, defines, rather, gtadjto, unreliable, andor, absentminded...|
|[spend, playing, computer, games, form, escapism, playing, happy, stop, wasting, playing, games, ...|
|[selfconfidence, great, terms, personality, ability, academic, pretty, happy, look, mirror, see, ...|
|[friends, speaking, food, server, sometimes, repeat, resort, pointing, t

Create pos-tags n-grams that correspond to words n-grams

In [20]:
#Merge POS tags as just one string to be able to take it as a document in the Spark NLP Pipeline
from pyspark.sql import functions as F
pos_as_string = F.udf(lambda x: ' '.join(x), T.StringType())
processed_data = processed_data.withColumn('finished_pos', pos_as_string(F.col('finished_pos')))

In [21]:
processed_data.show(3)

+------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos|
+------+--------------------+--------------------+--------------------+--------------------+
|1to3a6|can people sugges...|[suggest, courses...|[can, people, sug...|MD NNS VB NNS PRP...|
|21d7io|ive already come ...|[already, come, l...|[ive, already, co...|JJ RB VBP DT JJ N...|
|2tx2ga|today i accomplis...|[accomplished, ho...|[today, i, accomp...|NN NNP VBD IN DT ...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



New pipeline for pos-tags

In [22]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')
     
    
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) \

pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

processed_data = pos_pipeline.fit(processed_data).transform(processed_data)

In [23]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams']

In [24]:
processed_data.show(1)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1to3a6|can people sugges...|[suggest, courses...|[can, people, sug...|[MD, NNS, VB, NNS...|[MD, NNS, VB, NNS...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [25]:
def filter_unigrams(finished_unigrams, finished_pos):
    '''Filters individual words based on their POS tag'''
    return [word for word, pos in zip(finished_unigrams, finished_pos)
            if pos in ['JJ', 'NN', 'NNS', 'NNPS', 'VB']]

udf_filter_unigrams = F.udf(filter_unigrams, T.ArrayType(T.StringType()))

In [26]:
processed_data = processed_data.withColumn('filtered_unigrams_by_pos', udf_filter_unigrams(
                                                   F.col('finished_unigrams'),
                                                   F.col('finished_pos')))

processed_data.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|filtered_unigrams_by_pos|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+
|1to3a6|can people sugges...|[suggest, courses...|[can, people, sug...|[MD, NNS, VB, NNS...|[MD, NNS, VB, NNS...|    [courses, one, qu...|
|21d7io|ive already come ...|[already, come, l...|[ive, already, co...|[JJ, RB, VBP, DT,...|[JJ, RB, VBP, DT,...|    [already, shy, bo...|
|2tx2ga|today i accomplis...|[accomplished, ho...|[today, i, accomp...|[NN, NNP, VBD, IN...|[NN, NNP, VBD, IN...|    [accomplished, aw...|
|3dgbl9|hey guys so it se...|[guys, seems, fla...|[hey, guys, so, i...|[UH, NNS, IN, PRP...|[UH, NNS, IN, PRP...|    [seems, fickle, g...|
|4a6bsf|i spend a lot of ..

In [27]:
def filter_pos_ngrams(finished_ngrams, finished_pos_tags):
    return [word for word, pos in zip(finished_ngrams, finished_pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_ngrams = F.udf(filter_pos_ngrams, T.ArrayType(T.StringType()))

In [28]:
processed_data = processed_data.withColumn('filtered_ngrams_by_pos',
                       udf_filter_pos_ngrams(F.col('finished_ngrams'),
                                             F.col('finished_pos_ngrams')))

In [29]:
processed_data.show(5)

+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+----------------------+
|    id|        cleaned_text|   finished_unigrams|     finished_ngrams|        finished_pos| finished_pos_ngrams|filtered_unigrams_by_pos|filtered_ngrams_by_pos|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+----------------------+
|1to3a6|can people sugges...|[suggest, courses...|[can, people, sug...|[MD, NNS, VB, NNS...|[MD, NNS, VB, NNS...|    [courses, one, qu...|  [suggest_courses,...|
|21d7io|ive already come ...|[already, come, l...|[ive, already, co...|[JJ, RB, VBP, DT,...|[JJ, RB, VBP, DT,...|    [already, shy, bo...|  [long_way, super_...|
|2tx2ga|today i accomplis...|[accomplished, ho...|[today, i, accomp...|[NN, NNP, VBD, IN...|[NN, NNP, VBD, IN...|    [accomplished, aw...|  [hours_worth, fir...|
|3dgbl9|hey guys so it se...

In [30]:
processed_data.select('id','filtered_ngrams_by_pos').show(10, truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|    id|                                                                              filtered_ngrams_by_pos|
+------+----------------------------------------------------------------------------------------------------+
|1to3a6|[suggest_courses, do_quick, quick_courses, few_hours, improve_ones, ones_self, self_worth, worth_...|
|21d7io|[long_way, super_shy, shy_borderline, borderline_shutin, shutin_guy, obvious_im, meet_people, peo...|
|2tx2ga|[hours_worth, first_hour, hour_biology, biology_lab, hour_lecture, algorithms_book, cs_course, co...|
|3dgbl9|[flake_urban, urban_dictionary, dictionary_defines, be_unreliable, unreliable_andor, flighty_fick...|
|4a6bsf|                                                  [computer_games, things_im, be_happy, easy_option]|
|4t19a7|[academic_stuff, stuff_im, ugly_face, other_areas, low_selfesteem, selfesteem_due, looks_overwhel...|
|5l87qp|  

In [31]:
processed_data.select('id','filtered_unigrams_by_pos').show(10, truncate=100)

+------+----------------------------------------------------------------------------------------------------+
|    id|                                                                            filtered_unigrams_by_pos|
+------+----------------------------------------------------------------------------------------------------+
|1to3a6|[courses, one, quick, hours, improve, ones, couple, first, save, serving, course, takes, hours, s...|
|21d7io|[already, shy, borderline, highschool, obvious, boring, actually, boring, games, bitter, asshole,...|
|2tx2ga|[accomplished, awake, apart, lab, worthwhile, open, hardly, now, type, enter, code, online, late,...|
|3dgbl9|[seems, fickle, gtgenerally, unresponsible, now, solid, trying, find, real, absentminded, flighty...|
|4a6bsf|                                          [playing, games, escapism, happy, stop, easy, turn, break]|
|4t19a7|[terms, academic, mirror, ugly, suddenly, areas, gone, low, due, putting, mental, say, depressed,...|
|5l87qp|  

In [32]:
processed_data.columns

['id',
 'cleaned_text',
 'finished_unigrams',
 'finished_ngrams',
 'finished_pos',
 'finished_pos_ngrams',
 'filtered_unigrams_by_pos',
 'filtered_ngrams_by_pos']

Create one column merging unigrams and ngrams

In [33]:
from pyspark.sql.functions import concat
processed_data = processed_data.withColumn('final', concat(F.col('filtered_unigrams_by_pos'), F.col('filtered_ngrams_by_pos')))

In [34]:
processed_data.select('final').show(50, truncate= 100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               final|
+----------------------------------------------------------------------------------------------------+
|[courses, one, quick, hours, improve, ones, couple, first, save, serving, course, takes, hours, s...|
|[already, shy, borderline, highschool, obvious, boring, actually, boring, games, bitter, asshole,...|
|[accomplished, awake, apart, lab, worthwhile, open, hardly, now, type, enter, code, online, late,...|
|[seems, fickle, gtgenerally, unresponsible, now, solid, trying, find, real, absentminded, flighty...|
|[playing, games, escapism, happy, stop, easy, turn, break, computer_games, things_im, be_happy, e...|
|[terms, academic, mirror, ugly, suddenly, areas, gone, low, due, putting, mental, say, depressed,...|
|[speaking, sometimes, tried, sound, angry, commutation, have_problem, fo